# Differentially Expressed Proteins 
Statistical rigor is important when making claims about data--especially when human health gets involved. This snippet of code singles out data that is statistically "safe", that is, it weeds out the bad data that doesn't meet common assumptions for statistical tests between two populations.

Particularly, this code selects data for two populations that:

1.   Have comparable variances (as measured by an F-test with a p-value greater than 0.05)
2.   Are normally distributed (as measured by a shapro-wilk test with a p-value greater than 0.05)

*Results:*


*   The raw data contained expression levels for 6,794 proteins in healthy and cancerous ovary tissue
*   This code narrows the number of comparable proteins to 1458
*   This number is about 1/3 as large when a bonferroni multiple-testing correction is not used for the variance and normality tests (i.e. there are potentially many spurious non-normal and non-similar variances without bonferroni correction)







## Prepare Data

In [0]:
alpha = 0.05

In [0]:
# import libraries

import pandas as pd
import pickle
import numpy as np
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

In [0]:
# prepare notebook for uploading data

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# input file is "proteinGroups.tsv", open as a pandas dataframe

inputPath = "proteinGroups.txt"
df_raw = pd.read_csv(inputPath, sep="\t");

df_raw.shape[0]; # there are originally 6,794 proteins being considered

proteinIndices=df_raw[df_raw.columns[0]]
proteinDict={}
counter=1
for i in range(len(proteinIndices)-1):
  proteinDict[proteinIndices[i]]=df_raw.iat[counter,4]
  counter=counter+1

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (555,556) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# select just the LFQ-transformed data

df = df_raw.set_index("Protein IDs", drop = False)
df = df.loc[:, df.columns.str.startswith('LFQ')]

# columns represent different transformations of samples/replciates,
# rows represent different proteins (and are labled as such)


In [0]:
# make a list of the column names that represent healthy / cancerous samples

healthyColumnNames = []
cancerColumnNames = []
for column in df.columns:
    if "_NM" in column:
        healthyColumnNames.append(column)
    else:
        cancerColumnNames.append(column)

## Examine Normality
check to see that both the data for for each protein for the healthy and cancerous tissue are normally distributed.

*   Perform a separate shapiro-wilk test for 1) the healthy and 2) cancerous samples for each protein, 
*   Ignore "bad" replicates (replicates close to 0).
*   If ignoring bad replicates causes the assumptions of the Shapiro-Wilk-test to not be met, assign a p-value of 0
*   Save the p-value for each test



In [0]:
# 1) HEALTHY tissue samples:

shapiro_healthy = [] # initialize a list to store shapiro-test p-values
for index, row in df.iterrows(): # perform the test for each protein (row)
    dataList = row[healthyColumnNames]
    # ignore replicates that have near-zero values (i.e. "bad" data)
    cleanDataList = np.where(np.isclose(dataList,0), np.nan, dataList)
    cleanDataList = [x for x in cleanDataList if str(x) != 'nan']
    try:
        shapiro = stats.shapiro(cleanDataList)
        shapiro_healthy.append(shapiro[1]) # save just the p-value
    except ValueError: # if the assumptions of s.w. test are not met:
        shapiro_healthy.append(0)

        
# do a bonferroni correction on the p-values
shapiro_healthy_corrected = multipletests(shapiro_healthy, alpha, method='bonferroni')
# attach new p-values to the dataframe
df["shapiro_healthy"] = shapiro_healthy_corrected[1]

In [0]:
# 1) CANCER tissue samples:

shapiro_cancer = []
for index, row in df.iterrows():
    dataList = row[cancerColumnNames]
    cleanDataList = np.where(np.isclose(dataList,0), np.nan, dataList)
    cleanDataList = [x for x in cleanDataList if str(x) != 'nan']
    try:
        shapiro = stats.shapiro(cleanDataList)
        shapiro_cancer.append(shapiro[1])
    except ValueError:
        shapiro_cancer.append(0)

# do a bonferroni correction on the p-values
shapiro_cancer_corrected = multipletests(shapiro_cancer, alpha, method='bonferroni')
# attach new p-values to the dataframe
df["shapiro_cancer"] = shapiro_cancer_corrected[1]

## Trim non-normal data

In [0]:
# trim low p-vals

numProteinsA = len(df.index.values)
df = df[df["shapiro_healthy"] > alpha]
numProteinsB = len(df.index.values)
df = df[df["shapiro_cancer"] > alpha]
numProteinsC = len(df.index.values)

print("number of proteins before any trimming: " + str(numProteinsA))
print("number of proteins after removing non-normal healthy tissue: " + str(numProteinsB))
print("number of proteins after moving non-normal cancerous tissue: " + str(numProteinsC))

ndp = open("normallyDistributedProteins.pickle", "wb")
pickle.dump(df.index.values, ndp)



number of proteins before any trimming: 6794
number of proteins after removing non-normal healthy tissue: 3102
number of proteins after moving non-normal cancerous tissue: 2507


## Examine Variances
Both populations (healthy vs cancer, for a given protein) must have relatively equal variances in order to compare their means. Test variance using F-tests:

*   Perform an F-test for each protein, 
*   Ignore "bad" replicates (replicates close to 0).
*   If ignoring bad replicates causes the assumptions of the F-test to not be met, assign a p-value of 0
*   Save the p-value for each F-test

*This test is highly sensitive to non-normal samples, and therefore must be performed after the non-normal proteins have been removed (which is what the previous cell does)*

In [0]:
# an f-test was performed for each protein

# f_p_val = [] # initialize a list to store individual f-test p-values
# for index, row in df.iterrows(): # perform a test for every protein (row)
    
#     # separate the two populations
#     healthyList = row[healthyColumnNames]
#     cancerList = row[cancerColumnNames]

#     # calculate degrees of freedom as the population size - 1
#     df1 = (len(healthyList) - 1)
#     df2 = (len(cancerList) - 1)

#     # ignore data that is close to 0 when performing the test
#     cleanHealthyList = np.where(np.isclose(healthyList,0), np.nan, healthyList)
#     cleanCancerList = np.where(np.isclose(cancerList,0), np.nan, cancerList)
#     cleanHealthyList = [x for x in cleanHealthyList if str(x) != 'nan']
#     cleanCancerList = [x for x in cleanCancerList if str(x) != 'nan']
  
#     # calculate the p-value and save in list
#     try:
#         # calculate the F value
#         F = (np.var(cleanHealthyList) / np.var(cleanCancerList))
#         if str(F) != "nan" and F > 0.1:
#             p_val = stats.f.cdf(F, df1, df2)
#             f_p_val.append(p_val)
#         else:
#             f_p_val.append(0)
#     except: # when the assumptions of the test are not met:
#         f_p_val.append(0) # do not consider the data normally distributed 

# # attach new p-values to the dataframe
# # do a bonferroni correction on the p-values
# f_p_adjusted = multipletests(f_p_val, alpha, method='bonferroni')
# # attach new p-values to the dataframe
# df["f_p_val"] = f_p_adjusted[1]
# # ftest = open("fTestResults.pickle", "wb")
# # pickle.dump(df, f_p_adjusted[1])


# fPValDict = dict(zip(df.index.values, f_p_val))
# print(fPValDict)
# import pickle
# cr = open("fTestPVals.pickle", "wb")
# pickle.dump(fPValDict, cr)


## Trim non-equal variances
Discard all proteins that did not meet the assumptions of normality and equal variance between healthy and cancerous tissue

In [0]:

# df = df[df["f_p_val"] > alpha]
# numProteinsD = len(df.index.values)

# comparableRows = df.index.values

# import pickle
# cr = open("comparableRows.pickle", "wb")
# pickle.dump(comparableRows, cr)


# print("number of proteins after moving non-same variance proteins: " + str(numProteinsD))
# print("Final number of comparable proteins: " + str(len(comparableRows)))

## Perform the T-test

*   Two-sampled
*   Two-sided (multiply output p-value by two)
*   Bonferonni Correction (adjust 0.05 down according to the number of tests)



In [0]:
# do the two-sample t-test

tTestPVals = []
for index, row in df.iterrows():
    # grab the two "populations" for this protein
    healthyList = row[healthyColumnNames]
    cancerList = row[cancerColumnNames]
    
    # remove bad data
    cleanHealthyList = np.where(np.isclose(healthyList,0), np.nan, healthyList)
    cleanHealthyList = [x for x in cleanHealthyList if str(x) != 'nan']
    cleanCancerList = np.where(np.isclose(cancerList,0), np.nan, cancerList)
    cleanCancerList = [x for x in cleanCancerList if str(x) != 'nan']
    
    try:
        newPVal = stats.ttest_ind(cleanHealthyList, cleanCancerList, equal_var = False)
        tTestPVals.append(newPVal[1] * 2) # save just the p-value
    except ValueError: # if the assumptions of s.w. test are not met:
        tTestPVals.append(1)


    
    
# attach new p-values to the dataframe
# do a bonferroni correction on the p-values
t_p_adjusted = multipletests(tTestPVals, alpha, method='bonferroni')
# attach new p-values to the dataframe
df["t_p_val"] = t_p_adjusted[1]


In [0]:

abundanceDF = open("abundanceDF.pickle", "wb")
pickle.dump(df,abundanceDF)

In [0]:
df.shape

(2507, 62)

## Grab proteins that are signficantly different according to the t-test

In [0]:
df_diff_proteins = df[df["t_p_val"] < alpha]
df_noDiff_proteins = df[df["t_p_val"] > alpha]

differentProteins = df_diff_proteins.index.values
notDifferentProteins = df_noDiff_proteins.index.values

# check to make sure the lists are mutually exclusive
for protein in differentProteins:
  if protein in notDifferentProteins:
    print(protein)
    
for protein in notDifferentProteins:
  if protein in differentProteins:
    print(protein)
    
# print the lengths
print("Number of proteins that were significantly different: " + str(len(differentProteins)))
print("Number of proteins that were NOT significantly different: " + str(len(notDifferentProteins)))

print("lists of protein names are available as \"differentProteins.pickle\" and \"notDifferentProteins.pickle\"")
print("download those files if you dont' want to re-run this each time you need to generate those lists")

# save/download the proteins
import pickle
diffOut = open("diffProteins.pickle", "wb")
pickle.dump(differentProteins, diffOut)
noDiffOut = open("notDiffProteins.pickle", "wb")
pickle.dump(notDifferentProteins, noDiffOut)


# save the p-values
pVals = df["t_p_val"]
proteinNames = df.index.values
nameToPVal = dict(zip(proteinNames, pVals))

pValsOut = open("ProteinNamesToPValsDictionary.pickle", "wb")
pickle.dump(nameToPVal,pValsOut)




Number of proteins that were significantly different: 634
Number of proteins that were NOT significantly different: 1873
lists of protein names are available as "differentProteins.pickle" and "notDifferentProteins.pickle"
download those files if you dont' want to re-run this each time you need to generate those lists


#### Store the ACC Coefficients

In [0]:
# how to read in the pickle files:

newDiffProteins = pickle.load(open("diffProteins.pickle", "rb")) # update this to reflect the path in your computer
newNoDiffProteins = pickle.load(open("notDiffProteins.pickle", "rb")) # update this to reflect the path in your computer



print(len(newDiffProteins))
print(len(newNoDiffProteins))
# print(newDiffProteins)


for protein in newDiffProteins:
  if "Q92878" in protein:
    print("rad50")
if "Q92878" in newNoDiffProteins:
  pritn("noDiff")

# for i in newDiffProteins:
#   print(proteinDict[i])

634
1873
rad50
